Stage A — NASA (Canonical Degradation)
판단 근거
	•	NASAmatadata.csv는 메타 정보
	•	실제 열화는 각 discharge 파일의 Capacity / SOH
	•	지금 단계에서는 RUL 또는 Capacity 기반 Δstate만 쓰면 충분

In [4]:
# 코드 (Stage A 정규화)
import pandas as pd

# NASA metadata
nasa_meta = pd.read_csv("../data_csv/NASAmetadata.csv")

# 필요한 컬럼만
nasa_meta = nasa_meta[["battery_id", "test_id", "uid", "filename"]]


def load_nasa_cell(file_path, battery_id):
    df = pd.read_csv(file_path)
    
    # 예시 가정: Capacity 컬럼 존재
    df_out = pd.DataFrame({
        "asset_id": battery_id,
        "t_index": range(len(df)),
        "state_value": df["Capacity"]
    })
    return df_out


liBattery (Real-world Noise)
	•	battery_id
	•	Capacity
	•	시간 정보 없음 → 행 순서 = t_index

In [5]:
# 코드 (Stage B 정규화)
li_df = pd.read_csv("../data_csv/liBattery_Data_Cleaned.csv")

li_core = (
    li_df
    .sort_values(["battery_id", "uid"])
    .assign(
        asset_id=lambda x: x["battery_id"],
        t_index=lambda x: x.groupby("battery_id").cumcount(),
        state_value=lambda x: x["Capacity"]
    )
    [["asset_id", "t_index", "state_value"]]
)

li_core.head()

,asset_id,t_index,state_value
5043,5,0,0.810030
5044,5,1,1.049593
5045,5,2,1.049593
5046,5,3,1.046029
5047,5,4,1.046029


Stage C-1 — Synthetic Degradation 
	•	battery_id
	•	time
	•	rul ← 이미 열화 결과

In [6]:
# 코드 (Stage C-1)
synth_df = pd.read_csv("../data_csv/battery_degradation.csv")

synth_core = (
    synth_df
    .sort_values(["battery_id", "time"])
    .assign(
        asset_id=lambda x: x["battery_id"],
        t_index=lambda x: x.groupby("battery_id").cumcount(),
        state_value=lambda x: x["rul"]  # RUL → 열화 상태
    )
    [["asset_id", "t_index", "state_value"]]
)

synth_core.head()

,asset_id,t_index,state_value
0,B0005,0,2820.390
1,B0005,1,2811.062
2,B0005,2,2800.875
3,B0005,3,2791.453
4,B0005,4,2782.078


Stage C-2 — EV Synthetic 
	•	battery_id
	•	charge_cycles
	•	capacity_kWh

In [7]:
# 코드 (Stage C-2)
ev_synth = pd.read_csv("../data_csv/ev_battery_synth.csv")

ev_synth_core = (
    ev_synth
    .sort_values(["battery_id", "charge_cycles"])
    .assign(
        asset_id=lambda x: x["battery_id"],
        t_index=lambda x: x["charge_cycles"],
        state_value=lambda x: x["capacity_kWh"]
    )
    [["asset_id", "t_index", "state_value"]]
)

ev_synth_core.head()

,asset_id,t_index,state_value
0,BAT000001,1524.0,65.54
1,BAT000002,1072.0,NaN
2,BAT000003,807.0,99.99
3,BAT000004,681.0,85.96
4,BAT000005,476.0,69.85
